# Exploratory analysis for training an embedding model

In a parallel folder, we trained a masked geospatial model to yield "embeddings"
for regions. But they have the problme of not being perm-invariant WRT the ordering
of the inputs. Material in this folder seeks to ake those embddings perm-invariant.

We do this by applying a NN layer that computes a weighed sum of input vectors, where
the weights are learned form the features themselves. 

That model will be trained using contrastive loss.

And contrastive loss depends on an independent definition of whether a pair of regions
is similar or dissimilar. THAT is what I'm looking into here. 
Specifially I hope to define some feature vector for a region such that "similar" regions
have similar feature vectors. 

My first attempt at such a feature vector is one that encodes the coverage of different
geospatial entity types in a region. So here I'm going to compute that for a bunch 
of regions andsee how the results cluster. 



## Processing Setup

In [ ]:
# Google colab setup
# import os
# from google.colab import drive
# drive.mount('/content/drive')
# project_home = '/content/drive/MyDrive/Projects/verge'
# os.chdir(project_home)

In [ ]:
# Local processing setup
project_home = '..'

## Code

In [ ]:
import pandas as pd
import numpy as np
import geopandas

In [ ]:
# The name of the ROI to use.
roi_name = 'ne-dev'

# The name of the general-purpose data directory.
data_home = '%s/data' % (project_home)

# The name of the ROI-specific data directory.
roi_home = '%s/data/%s' % (project_home, roi_name)


In [ ]:
# Load labels
fname = "%s/labels.csv" % data_home
labels = pd.read_csv(fname)
labels.head(3)

In [ ]:
# Get a list of tiles.
fname = '%s/tile_info.csv' % roi_home
tile_info = pd.read_csv(fname)
print('%d tiles' % len(tile_info))
tile_info.head(3)

In [ ]:
# Make a lookup table giving each tile's lon/lat as a function of
# its path name components.
tile_info['key'] = [
    z['fname'][-21:]
    for z in tile_info.to_dict('records')
]
tile_coord_lookup = {
    z['fname'][-21:]: (z['center_lon'], z['center_lat'])
    for z in tile_info.to_dict('records')
}

In [ ]:
tile_coord_lookup

In [ ]:
# Get a list of the tile file names.
import glob
globstring = '%s/tiles/*/*.pq' % roi_home
tile_fnames = glob.glob(globstring)
tile_fnames[:3]

In [ ]:
# Get feature vectors for each tile.

feature_vectors = []
center_lons = []
center_lats = []

for k, tile_fname in enumerate(tile_fnames):

    if k % 50 == 0:
        print('handling tile %d / %d' % (k, len(tile_fnames)))

    try:
        tile_key = tile_fname[-21:]
        coords = tile_coord_lookup[tile_key]
        center_lon = coords[0]
        center_lat = coords[1]
    except:
        print('bad tile key: %s' % tile_key)
        continue

    # Read the tile data.
    tile = geopandas.read_parquet(tile_fname)

    # Make a label column that matches the format of the "labels" data frame.
    tile['full_label'] = tile['category'] + ' : ' + tile['label']

    # Get a tally of the coverage of each type of thing.
    coverages = { z: 0.0 for z in labels['label'].values }
    
    for rec in tile.to_dict('records'):
        if rec['gtype'] == 'Polygon':
            numerator = rec['geometry'].area / 1000000.0
        elif rec['gtype'] == 'LineString':
            numerator = rec['geometry'].length / 1000.0
        else:
            numerator = 1.0
    
        label = rec['full_label']
        coverages[label] += numerator

    # Get a feature vector and add it to the list.
    v = np.array([coverages[z] for z in sorted(labels['label'].values)])
    feature_vectors.append(v)
    center_lons.append(center_lon)
    center_lats.append(center_lat)
    

In [ ]:
features = np.vstack(feature_vectors)
print('feature matrix shape', features.shape)

In [ ]:
# Normalize the rows and columns.
m = features.mean(axis=0)
s = features.std(axis=0)
zzz = (features - m) / (s + 0.0001)

## TSNE plot of feature vectors

In [ ]:
# Make a TSNE plot of the scaled features
from sklearn.manifold import TSNE
tsne = TSNE()
xy = tsne.fit_transform(zzz)

## Run a DBSCAN clustering on the feature vectors.

In [ ]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=30)
model.fit(feature_vectors)
print('%d clusters' % len(set(model.labels_)))

In [ ]:
import plotly
from plotly.subplots import make_subplots
from plotly.graph_objects import Scatter

colors = [
    "#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00",
    "#ffff33", "#a65628", "#f781bf", "#999999", "#66c2a5",
    "#fc8d62", "#8da0cb", "#e78ac3", "#a6d854", "#ffd92f",
    "#e5c494", "#b3b3b3", "#1b9e77", "#d95f02", "#7570b3",
    "#e7298a", "#66a61e", "#e6ab02", "#a6761d", "#666666",
    "#8dd3c7", "#ffffb3", "#bebada", "#fb8072", "#80b1d3",
    "#fdb462", "#b3de69", "#fccde5", "#d9d9d9", "#bc80bd",
    "#ccebc5", "#ffed6f", "#a1dab4", "#41b6c4", "#2c7fb8"
]

labels = model.labels_
ulabels = set(labels)

fig = make_subplots(rows=1, cols=1)
for k, ulabel in enumerate(ulabels):
    iok = labels == ulabel
    tr = Scatter(x=xy[iok,0], y=xy[iok,1], mode='markers', 
                 marker={'color': colors[k%40]})
    fig.append_trace(tr, 1, 1)
        
lo = fig['layout']
lo['width'] = 800
lo['height'] = 500
fig


## Make a map showing the locations of all elements of a cluster

In [ ]:
import folium

label = 3
iok = labels == label
coords = list(zip(np.array(center_lons)[iok], np.array(center_lats)[iok]))
# coords = list(zip(np.array(center_lons), np.array(center_lats)))

m = folium.Map(location=[coords[0][1], coords[0][0]], zoom_start=5)

# Add solid circle markers for each coordinate
for lon, lat in coords:
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,               # Small circle
        color='black',          # Circle border color
        fill=True,
        fill_color='green',     # Solid fill color
        fill_opacity=1.0        # Fully opaque
    ).add_to(m)

m


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
plt.imshow(zzz, cmap='viridis', aspect='auto')
plt.colorbar(label='Value')
plt.title('Basic Heatmap')
plt.xlabel('X axis')
plt.ylabel('Y axis')
plt.show()


In [ ]:
subarrays = []
for i in range(10):
    iok = labels == i
    subarrays.append(zzz[iok, :])
    subarrays.append(10 * np.ones((5, zzz.shape[1])))

ooo = np.vstack(subarrays)
ooo.shape

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
plt.imshow(ooo, cmap='viridis', aspect='auto')
plt.colorbar(label='Value')
plt.title('Rows grouped by cluster')
plt.xlabel('X axis')
plt.ylabel('Y axis')
plt.show()
